### Get the Ground Truth Distributions
First, we will get the ground truth distributions for the patches images and their aggregated images.

In [1]:
from src.segmentation.evaluation.detectron2_evaluator import Detectron2Evaluator
import os

# varaiables
dataset_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44"
images_dir_path = os.path.join(dataset_path, "v0.1")
coco_annotations_path = os.path.join(dataset_path, "annotations", "export_coco-instance_etaylor_cannabis_patches_test_26-04-2024_15-44-44_v0.1.json")
yolo_annotations_dir_path = os.path.join(dataset_path, "annotations", "yolo", "labels", "export_coco-instance_etaylor_cannabis_patches_test_26-04-2024_15-44-44_v0.1")

# get evalutor and get the ground truth boxes
detectron2_evaluator = Detectron2Evaluator(num_classes=3,coco_annotations_file_path=coco_annotations_path)
dataset_gt_boxes = detectron2_evaluator.get_annotations_for_dataset(images_directory=images_dir_path)

In [2]:
def calculate_trichome_distribution(annotations_dict):
    distribution_dict = {}
    image_distribution_dict = {}

    for image, patches in annotations_dict.items():
        image_distribution = {0: 0, 1: 0, 2: 0}
        distribution_dict[image] = {}

        for patch, annotations in patches.items():
            patch_distribution = {0: 0, 1: 0, 2: 0}

            for annotation in annotations:
                class_id = annotation['class_id']
                patch_distribution[class_id] += 1
                image_distribution[class_id] += 1

            distribution_dict[image][patch] = patch_distribution

        image_distribution_dict[image] = image_distribution

    return distribution_dict, image_distribution_dict

In [3]:
patch_distribution, image_distribution = calculate_trichome_distribution(dataset_gt_boxes)

In [4]:
# Print results
print("Patch Distribution:")
for image, patches in patch_distribution.items():
    print(f"\n{image}:")
    for patch, dist in patches.items():
        print(f"  {patch}: {dist}")

print("\nImage Distribution:")
for image, dist in image_distribution.items():
    print(f"{image}: {dist}")

Patch Distribution:

IMG_0058:
  IMG_0058_p0.png: {0: 6, 1: 2, 2: 0}
  IMG_0058_p1.png: {0: 16, 1: 2, 2: 0}
  IMG_0058_p2.png: {0: 14, 1: 7, 2: 0}
  IMG_0058_p3.png: {0: 7, 1: 6, 2: 0}
  IMG_0058_p4.png: {0: 3, 1: 1, 2: 0}
  IMG_0058_p5.png: {0: 3, 1: 1, 2: 0}
  IMG_0058_p6.png: {0: 9, 1: 4, 2: 0}
  IMG_0058_p7.png: {0: 12, 1: 5, 2: 0}
  IMG_0058_p8.png: {0: 7, 1: 3, 2: 0}

IMG_1096:
  IMG_1096_p0.png: {0: 0, 1: 6, 2: 1}
  IMG_1096_p10.png: {0: 3, 1: 22, 2: 7}
  IMG_1096_p11.png: {0: 1, 1: 8, 2: 4}
  IMG_1096_p12.png: {0: 3, 1: 9, 2: 3}
  IMG_1096_p1.png: {0: 0, 1: 1, 2: 2}
  IMG_1096_p3.png: {0: 0, 1: 4, 2: 0}
  IMG_1096_p4.png: {0: 0, 1: 11, 2: 9}
  IMG_1096_p6.png: {0: 5, 1: 5, 2: 3}
  IMG_1096_p7.png: {0: 0, 1: 3, 2: 2}
  IMG_1096_p8.png: {0: 5, 1: 17, 2: 5}
  IMG_1096_p9.png: {0: 5, 1: 23, 2: 3}

IMG_2198:
  IMG_2198_p0.png: {0: 0, 1: 4, 2: 0}
  IMG_2198_p10.png: {0: 4, 1: 5, 2: 0}
  IMG_2198_p11.png: {0: 2, 1: 5, 2: 3}
  IMG_2198_p12.png: {0: 3, 1: 7, 2: 1}
  IMG_2198_p1.png: {0:

In [5]:
import pandas as pd
# build dataframe from the dicts distributions
images_annotations = []

for image_number, image_dist in image_distribution.items():
    clear_count = image_dist.get(0, 0)
    cloudy_count = image_dist.get(1, 0)
    amber_count = image_dist.get(2, 0)
    total_count = clear_count + cloudy_count + amber_count

    # Calculate normalized distribution
    if total_count > 0:
        clear_normalized = clear_count / total_count
        cloudy_normalized = cloudy_count / total_count
        amber_normalized = amber_count / total_count
    else:
        clear_normalized = cloudy_normalized = amber_normalized = 0

    images_annotations.append({
        "image_number": image_number,
        "clear": clear_count,
        "cloudy": cloudy_count,
        "amber": amber_count,
        "clear_norm": clear_normalized,
        "cloudy_norm": cloudy_normalized,
        "amber_norm": amber_normalized
    })
    
images_gt_df = pd.DataFrame(images_annotations)
images_gt_df

,image_number,clear,cloudy,amber,clear_norm,cloudy_norm,amber_norm
0,IMG_0058,77,31,0,0.712963,0.287037,0.000000
1,IMG_1096,22,109,39,0.129412,0.641176,0.229412
2,IMG_2198,48,148,10,0.233010,0.718447,0.048544
3,IMG_1093,33,143,36,0.155660,0.674528,0.169811
4,IMG_0542,36,48,0,0.428571,0.571429,0.000000
5,IMG_2271,23,57,10,0.255556,0.633333,0.111111
6,IMG_0019,140,99,3,0.578512,0.409091,0.012397
7,IMG_1079,68,79,4,0.450331,0.523179,0.026490
8,IMG_1753,73,21,0,0.776596,0.223404,0.000000
9,IMG_2285,29,127,12,0.172619,0.755952,0.071429


In [6]:
# create patches ground truth dataframe
patches_annotations = []

for image_number, patches in patch_distribution.items():
    for patch_number, patch_dist in patches.items():
        clear_count = patch_dist.get(0, 0)
        cloudy_count = patch_dist.get(1, 0)
        amber_count = patch_dist.get(2, 0)
        total_count = clear_count + cloudy_count + amber_count

        # Calculate normalized distribution
        if total_count > 0:
            clear_normalized = clear_count / total_count
            cloudy_normalized = cloudy_count / total_count
            amber_normalized = amber_count / total_count
        else:
            clear_normalized = cloudy_normalized = amber_normalized = 0

        patches_annotations.append({
            "image_number": image_number,
            "patch_number": patch_number,
            "clear": clear_count,
            "cloudy": cloudy_count,
            "amber": amber_count,
            "clear_norm": clear_normalized,
            "cloudy_norm": cloudy_normalized,
            "amber_norm": amber_normalized
        })
    
patches_gt_df = pd.DataFrame(patches_annotations)
patches_gt_df.head()

,image_number,patch_number,clear,cloudy,amber,clear_norm,cloudy_norm,amber_norm
0,IMG_0058,IMG_0058_p0.png,6,2,0,0.750000,0.250000,0.0
1,IMG_0058,IMG_0058_p1.png,16,2,0,0.888889,0.111111,0.0
2,IMG_0058,IMG_0058_p2.png,14,7,0,0.666667,0.333333,0.0
3,IMG_0058,IMG_0058_p3.png,7,6,0,0.538462,0.461538,0.0
4,IMG_0058,IMG_0058_p4.png,3,1,0,0.750000,0.250000,0.0


### Prediction Models Trichomes Distributions
Here we will use the detection results from the previous experiment done for comparing detections models.
We will extract the bboxes from the detections of the models and use them to extract trichomes images for evaluation.
After extracting the trichomes images we will classify them using the Alexnet model and evalute the results.

In [7]:
# laod the pred boxes of each model
import json
import os
import numpy as np

# load the ultralytics models pred boxes
ultralytics_models_pred_boxes_saveing_path = os.path.join("/home/etaylor/code_projects/thesis/data/models_scores", "ultralytics_models_pred_boxes_result_22_05_2024.json")
detectron2_models_pred_boxes_saveing_path = os.path.join("/home/etaylor/code_projects/thesis/data/models_scores", "detectron2_models_pred_boxes_result_22_05_2024.json")

with open(ultralytics_models_pred_boxes_saveing_path, "r") as f:
    ultralytics_models_pred_boxes = json.load(f)
    
with open(detectron2_models_pred_boxes_saveing_path, "r") as f:
    detectron2_models_pred_boxes = json.load(f)

### Create Classification Dataset from Detections
In this part we will use the detection from the model compared in the last comparison and create dataset that will be used to infrence with the alexnet model.

In [22]:
import os
import cv2

def create_trichome_dataset_from_detections(detection_dict, images_directory, output_directory, extend_percentage=0):
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Process each image and crop trichomes
    for image_number, patches_detections in detection_dict.items():
        for patch_file_name, detections in patches_detections.items():
            # Extract patch number from the filename
            patch_number = patch_file_name.split('_p')[1].split('.')[0]
            
            image_path = os.path.join(images_directory, patch_file_name)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Image {image_path} not found or unable to read.")
                continue

            for i, detection in enumerate(detections):
                bbox = detection['bbox']

                x_min, y_min, x_max, y_max = map(int, bbox)

                # Calculate the width and height of the bounding box
                width = x_max - x_min
                height = y_max - y_min

                # Calculate the extension for width and height
                width_extension = int(width * extend_percentage)
                height_extension = int(height * extend_percentage)

                # Extend the bounding box coordinates
                x_min = max(x_min - width_extension, 0)
                y_min = max(y_min - height_extension, 0)
                x_max = min(x_max + width_extension, image.shape[1])
                y_max = min(y_max + height_extension, image.shape[0])

                # Crop the trichome with extended bounding box
                trichome_crop = image[y_min:y_max, x_min:x_max]

                # Save the cropped image with the desired naming convention including patch number
                output_path = os.path.join(output_directory, f"{image_number}_p{patch_number}_trichome_{i}.png")
                cv2.imwrite(output_path, trichome_crop)

                print(f"Cropped trichome saved at: {output_path}")

    print(f"Dataset creation completed. Images saved in {output_directory}")

In [ ]:
detectron2_models_pred_boxes['faster_rcnn_R_50_DC5_1x']

In [23]:
# creating the trichomes dataset of the faster_rcnn_R_50_DC5_1x mode
saving_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x"

create_trichome_dataset_from_detections(detection_dict=detectron2_models_pred_boxes['faster_rcnn_R_50_DC5_1x'],
                                        images_directory=images_dir_path,
                                        output_directory=saving_path,
                                        extend_percentage=0.1)

Cropped trichome saved at: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x/IMG_1096_p10_trichome_0.png
Cropped trichome saved at: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x/IMG_1096_p10_trichome_1.png
Cropped trichome saved at: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x/IMG_1096_p10_trichome_2.png
Cropped trichome saved at: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x/IMG_1096_p10_trichome_3.png
Cropped trichome saved at: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x/IMG_1096

### Train Alexnet Model
Now we will fine tune the alexnet model that we will use for the classification task

In [8]:
from fastai.vision.all import *
from fastai.vision import *

data = {
        'train': '/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/trichome_dataset_01',
        'test': '/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/trichome_dataset_01',
    }

# define train metrics
precision_macro_fastai = Precision(average='macro')
recall_macro_fastai = Recall(average='macro')
roc_auc_fastai = RocAuc()


# transformation and image space conversion
def custom_transform(size):
    return Resize(size, method='pad', pad_mode='zeros')

class RGB2HSV(Transform):
    def encodes(self, img: PILImage): 
        return rgb2hsv(img)
    
    
global_item_tfms=custom_transform(size=128),  # Resize and HSV transform
global_batch_tfms=[
    RGB2HSV(),
    *aug_transforms(size=128, flip_vert=True, max_rotate=10),
    Brightness(max_lighting=0.2, p=0.75),
    Contrast(max_lighting=0.2, p=0.75),
]

dls = ImageDataLoaders.from_folder(
        path=data['train'],
        item_tfms=global_item_tfms,
        batch_tfms=global_batch_tfms,
        bs=16,
        valid_pct=0.25
    )

model = vision_learner(
            dls=dls,
            arch=models.alexnet,
            metrics=[error_rate, precision_macro_fastai, recall_macro_fastai, roc_auc_fastai]
        )

model.fine_tune(epochs=100)

epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,roc_auc_score,time
0,1.562871,1.116674,0.458867,0.557624,0.513517,0.755080,00:06


epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,roc_auc_score,time
0,1.111726,0.822226,0.363803,0.651030,0.589478,0.812715,00:03
1,1.028280,0.771384,0.332724,0.664672,0.621920,0.826882,00:02
2,0.981307,0.732131,0.308958,0.691874,0.686212,0.845278,00:02
3,0.951773,0.696457,0.290676,0.702461,0.702498,0.854495,00:02
4,0.905506,0.668165,0.285192,0.712449,0.716218,0.861481,00:02
5,0.867581,0.696895,0.288848,0.727249,0.710823,0.861511,00:02
6,0.812123,0.640685,0.257770,0.750013,0.725635,0.872152,00:02
7,0.815706,0.656867,0.272395,0.746118,0.717904,0.863809,00:02
8,0.763217,0.645630,0.270567,0.761083,0.692654,0.866856,00:02
9,0.750255,0.629563,0.270567,0.725668,0.704581,0.874757,00:02


In [ ]:
from collections import defaultdict
import os
import pandas as pd
from fastai.vision.all import *

saving_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x"

# Function to predict a single image
def predict_image(img_path, model):
    img = PILImage.create(img_path)
    pred_class, _, _ = model.predict(img)
    return pred_class

# Function to aggregate results for each patch
def aggregate_patch_results(predictions):
    return {cls: predictions.count(cls) for cls in set(predictions)}

# Dictionary to store predictions for each patch
patch_predictions = defaultdict(list)

# Predict each image and store results
for img_name in os.listdir(saving_path):
    if img_name.endswith('.png'):
        img_path = os.path.join(saving_path, img_name)
        pred_class = predict_image(img_path, model)
        
        # Extract patch name from image name
        patch_name = '_'.join(img_name.split('_')[:3])  # Assumes format like IMG_0019_p10_trichome_5.png
        
        patch_predictions[f"{patch_name}.png"].append(str(pred_class))

# Aggregate results for each patch
aggregated_results = {patch: aggregate_patch_results(preds) for patch, preds in patch_predictions.items()}

# Convert results to a DataFrame
results_df = pd.DataFrame.from_dict(aggregated_results, orient='index')
results_df = results_df.fillna(0).astype(int)  # Replace NaN with 0 and convert to int

# Reset the index, rename it, and add the patch number as a column
results_df = results_df.reset_index()
results_df = results_df.rename(columns={'index': 'Patch Number'})

# Ensure the 'Patch Number' column is the first column
cols = results_df.columns.tolist()
cols = ['Patch Number'] + [col for col in cols if col != 'Patch Number']
results_df = results_df[cols]

# Reset the index to regular numbers
results_df = results_df.reset_index(drop=True)

In [12]:
print(f"Length of results: {len(results_df)}")
# remove patch_number column from df

results_df.head()

Length of results: 108


,Patch Number,cloudy,clear,amber
0,IMG_1096_p10.png,18,8,9
1,IMG_1096_p11.png,10,2,8
2,IMG_1096_p1.png,1,1,3
3,IMG_1096_p12.png,16,3,4
4,IMG_1096_p3.png,1,4,1


In [18]:
import pandas as pd
import numpy as np

def normalize_patch_scores(df):
    # Create a copy of the original DataFrame
    result_df = df.copy()
    
    # Identify columns to normalize (exclude 'Patch Number')
    columns_to_normalize = [col for col in df.columns if col != 'Patch Number']
    
    # Calculate normalized scores
    normalized_scores = df[columns_to_normalize].div(df[columns_to_normalize].sum(axis=1), axis=0)
    
    # Add normalized scores to the result DataFrame with a 'norm_' prefix
    for column in normalized_scores.columns:
        result_df[f'{column}_norm'] = normalized_scores[column]
    
    return result_df

def aggregate_image_scores(df):
    # Extract image names from 'Patch Number' column
    df['image'] = df['patch_number'].str.split('_p').str[0]
    
    # Identify columns to aggregate (exclude 'Patch Number' and 'image')
    columns_to_aggregate = [col for col in df.columns if col not in ['Patch Number', 'image']]
    
    # Group by image and sum the scores
    image_scores = df.groupby('image')[columns_to_aggregate].sum()
    
    # Normalize the aggregated scores
    normalized_scores = image_scores.div(image_scores.sum(axis=1), axis=0)
    
    # Add normalized scores to the result DataFrame with a 'norm_' prefix
    for column in normalized_scores.columns:
        image_scores[f'{column}_norm'] = normalized_scores[column]
    
    return image_scores

# Normalize patch scores and add to the original DataFrame
patches_pred_df = normalize_patch_scores(results_df)

# Aggregate and normalize image scores
images_pred_df = aggregate_image_scores(results_df)


In [28]:
patches_pred_df.head()


,patch_number,cloudy,clear,amber,cloudy_norm,clear_norm,amber_norm
0,IMG_1096_p10.png,18,8,9,0.514286,0.228571,0.257143
1,IMG_1096_p11.png,10,2,8,0.500000,0.100000,0.400000
2,IMG_1096_p1.png,1,1,3,0.200000,0.200000,0.600000
3,IMG_1096_p12.png,16,3,4,0.695652,0.130435,0.173913
4,IMG_1096_p3.png,1,4,1,0.166667,0.666667,0.166667


In [27]:
patches_gt_df.head()

,image_number,patch_number,clear,cloudy,amber,clear_norm,cloudy_norm,amber_norm,patch_id
0,IMG_0058,IMG_0058_p0.png,6,2,0,0.750000,0.250000,0.0,IMG_0058_IMG_0058_p0
1,IMG_0058,IMG_0058_p1.png,16,2,0,0.888889,0.111111,0.0,IMG_0058_IMG_0058_p1
2,IMG_0058,IMG_0058_p2.png,14,7,0,0.666667,0.333333,0.0,IMG_0058_IMG_0058_p2
3,IMG_0058,IMG_0058_p3.png,7,6,0,0.538462,0.461538,0.0,IMG_0058_IMG_0058_p3
4,IMG_0058,IMG_0058_p4.png,3,1,0,0.750000,0.250000,0.0,IMG_0058_IMG_0058_p4


In [54]:
# get the records of the patch_number IMG_0058_p0.png from patches_gt_df
val_gt = patches_gt_df[patches_gt_df['patch_number'] == 'IMG_0058_p0.png']
val_pred = patches_pred_df[patches_pred_df['patch_number'] == 'IMG_0058_p0.png']



In [56]:
print(f"ground truth:\n {val_gt}")
print(f"prediction:\n {val_pred}")

ground truth:
   image_number     patch_number  clear  cloudy  amber  clear_norm  \
0     IMG_0058  IMG_0058_p0.png      6       2      0        0.75   

   cloudy_norm  amber_norm              patch_id  
0         0.25         0.0  IMG_0058_IMG_0058_p0  
prediction:
        patch_number  cloudy  clear  amber  cloudy_norm  clear_norm  amber_norm
98  IMG_0058_p0.png       5      4      0     0.555556    0.444444         0.0


### Evaluate Regression Scores

In [64]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0  # Exclude zero values to avoid division by zero
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100

def calculate_metrics(y_true, y_pred):
    # mse = mean_squared_error(y_true, y_pred)
    # rmse = np.sqrt(mse)
    # mae = mean_absolute_error(y_true, y_pred)
    # r2 = r2_score(y_true, y_pred)
    mape = calculate_mape(y_true, y_pred)
    return {'MAPE': mape}


def evaluate_patch_predictions(patches_gt_df, patches_pred_df, model_name):
    results = {"model_name": model_name}
    trichome_types = ['clear', 'cloudy', 'amber']

    patches_gt_df = patches_gt_df.set_index('patch_number')
    patches_pred_df = patches_pred_df.set_index('patch_number')

    merged_patches = pd.merge(patches_gt_df, patches_pred_df, 
                              left_index=True, right_index=True, 
                              suffixes=('_gt', '_pred'))
    
    # Analyze distribution of predictions
    pred_dist = merged_patches[[f'{t}_norm_pred' for t in trichome_types]]
    gt_dist = merged_patches[[f'{t}_norm_gt' for t in trichome_types]]

    # Plot distribution of predictions
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=pred_dist)
    plt.title(f'Distribution of Normalized Predictions - {model_name}')
    plt.ylabel('Proportion')
    plt.savefig(f'pred_distribution_{model_name}.png')
    plt.close()

    # Plot distribution of ground truth
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=gt_dist)
    plt.title(f'Distribution of Normalized Ground Truth - {model_name}')
    plt.ylabel('Proportion')
    plt.savefig(f'gt_distribution_{model_name}.png')
    plt.close()

    # Compute average proportions
    avg_pred = pred_dist.mean()
    avg_gt = gt_dist.mean()

    for t in trichome_types:
        results[f'Avg_Pred_{t}'] = avg_pred[f'{t}_norm_pred']
        results[f'Avg_GT_{t}'] = avg_gt[f'{t}_norm_gt']

    # Check if predictions sum to 1
    pred_sums = pred_dist.sum(axis=1)
    gt_sums = gt_dist.sum(axis=1)

    results['Pred_Sum_Mean'] = pred_sums.mean()
    results['Pred_Sum_Std'] = pred_sums.std()
    results['GT_Sum_Mean'] = gt_sums.mean()
    results['GT_Sum_Std'] = gt_sums.std()

    # Analyze errors
    for t in trichome_types:
        error = merged_patches[f'{t}_norm_pred'] - merged_patches[f'{t}_norm_gt']
        plt.figure(figsize=(10, 6))
        sns.scatterplot(x=merged_patches[f'{t}_norm_gt'], y=error)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.title(f'Error vs Ground Truth for {t} Trichomes')
        plt.xlabel('Ground Truth')
        plt.ylabel('Error (Predicted - Ground Truth)')
        plt.savefig(f'error_analysis_{t}_{model_name}.png')
        plt.close()

    return pd.DataFrame([results]), merged_patches

In [65]:
# Usage example:
patch_metrics, merged_patches = evaluate_patch_predictions(patches_gt_df, patches_pred_df, "Faster RCNN with Alexnet")

In [60]:
patch_metrics

,model_name,Patch clear MAPE,Patch cloudy MAPE,Patch amber MAPE,Patch Overall MAPE,Patch norm_clear MAPE,Patch norm_cloudy MAPE,Patch norm_amber MAPE,Patch norm_Overall MAPE
0,Faster RCNN with Alexnet,65.501524,63.757282,57.805463,63.365174,67.669777,40.925158,54.682494,53.80723


In [68]:
import matplotlib.pyplot as plt

def plot_prediction_vs_ground_truth(merged_patches, model_name):
    trichome_types = ['clear', 'cloudy', 'amber']
    score_types = ['norm_', '']
    
    for score_type in score_types:
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        fig.suptitle(f'{"Normalized" if score_type == "norm_" else "Non-normalized"} Prediction vs Ground Truth for {model_name}')
        
        for i, trichome in enumerate(trichome_types):
            ax = axes[i]
            gt = merged_patches[f'{trichome}_{score_type}gt']
            pred = merged_patches[f'{trichome}_{score_type}pred']
            
            ax.scatter(gt, pred, alpha=0.5)
            
            if score_type == 'norm_':
                ax.plot([0, 1], [0, 1], 'r--')  # diagonal line for normalized data
                ax.set_xlim(0, 1)
                ax.set_ylim(0, 1)
            else:
                max_val = max(gt.max(), pred.max())
                ax.plot([0, max_val], [0, max_val], 'r--')  # diagonal line for non-normalized data
                ax.set_xlim(0, max_val)
                ax.set_ylim(0, max_val)
            
            ax.set_xlabel('Ground Truth')
            ax.set_ylabel('Predicted')
            ax.set_title(f'{trichome.capitalize()} Trichomes')
        
        plt.tight_layout()
        plt.savefig(f'{"normalized" if score_type == "norm_" else "non_normalized"}_prediction_vs_ground_truth_{model_name}.png')
        plt.close()


In [69]:
# Use this function in your evaluate_patch_predictions function
plot_prediction_vs_ground_truth(merged_patches, "Faster RCNN with Alexnet")

In [70]:
import matplotlib.pyplot as plt
import numpy as np
import ternary

def plot_ternary_comparison(merged_patches, model_name):
    # Extract normalized ground truth and predicted values
    gt_clear = merged_patches['clear_norm_gt']
    gt_cloudy = merged_patches['cloudy_norm_gt']
    gt_amber = merged_patches['amber_norm_gt']
    
    pred_clear = merged_patches['clear_norm_pred']
    pred_cloudy = merged_patches['cloudy_norm_pred']
    pred_amber = merged_patches['amber_norm_pred']

    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(10, 8))
    tax = ternary.TernaryAxesSubplot(ax=ax, scale=1.0)

    # Plot ground truth points
    tax.scatter([(gt_clear[i], gt_cloudy[i], gt_amber[i]) for i in range(len(gt_clear))], 
                marker='o', color='blue', label='Ground Truth', alpha=0.5)

    # Plot predicted points
    tax.scatter([(pred_clear[i], pred_cloudy[i], pred_amber[i]) for i in range(len(pred_clear))], 
                marker='s', color='red', label='Predicted', alpha=0.5)

    # Customize the plot
    tax.boundary(linewidth=2.0)
    tax.gridlines(multiple=0.1, color="gray")
    tax.set_title(f"Ternary Plot of Trichome Types for {model_name}", fontsize=12)

    # Set labels
    tax.left_axis_label("Clear", fontsize=10)
    tax.right_axis_label("Cloudy", fontsize=10)
    tax.bottom_axis_label("Amber", fontsize=10)

    # Add legend
    ax.legend(loc='best')

    # Save the plot
    plt.tight_layout()
    plt.savefig(f'ternary_plot_{model_name}.png')
    plt.close()

In [71]:
plot_ternary_comparison(merged_patches, "Faster RCNN with Alexnet")

/tmp/ipykernel_2478556/3655797617.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tax.scatter([(gt_clear[i], gt_cloudy[i], gt_amber[i]) for i in range(len(gt_clear))],
/home/etaylor/.conda/envs/fastai_env/lib/python3.10/site-packages/ternary/plotting.py:148: UserWarning: No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
  ax.scatter(xs, ys, vmin=vmin, vmax=vmax, **kwargs)
/tmp/ipykernel_2478556/3655797617.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tax.scatter([(pred_clear[i], pred_cloudy[i], pred_amber[i]) for i in range(len(pred_clear))],


In [ ]:
# TODO: organize the predictions dataframes that it would be easier to compare the results
# TODO: align the format of the pred and gt dfs
# TODO: calc evaluation metrics
# TODO: add comparison for the other detection models and eval results